In [1]:
import transformers
import pandas as pd
import tensorflow as tf
import numpy as np
from transformers import TFDistilBertForSequenceClassification, TextClassificationPipeline, DistilBertTokenizerFast
from sklearn.model_selection import train_test_split

### for the import below: pip install datasets ###

from datasets import list_metrics, load_metric


2022-04-06 16:39:29.386549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-06 16:39:29.386587: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = TFDistilBertForSequenceClassification.from_pretrained('./models/.')

tokenizer = DistilBertTokenizerFast.from_pretrained('./models/tf_tokenizer/.')

2022-04-06 16:39:31.375047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-06 16:39:31.375076: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-06 16:39:31.375097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-schulma1): /proc/driver/nvidia/version does not exist
2022-04-06 16:39:31.375296: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 16:39:31.418789: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequen

In [3]:
pipe = TextClassificationPipeline(model = model, tokenizer = tokenizer, return_all_scores=True)

In [4]:
pipe("I hate the government restrictions")

[[{'label': 'LABEL_0', 'score': 0.22559571266174316},
  {'label': 'LABEL_1', 'score': 0.7744042873382568}]]

In [5]:
d = pd.read_excel('tweets_labeled.xlsx')
tweets = d['full_text'][:600]
labels = d['Label (critical: 1/neutral: 0)'][:600]
data = pd.DataFrame({'tweets': tweets, 'labels': labels})

data_train, data_test = train_test_split(data,test_size=0.2,random_state=1)

In [6]:
def predict_label(tweet):
    pred = pipe(tweet)
    if pred[0][0]['score'] > pred[0][1]['score']: # predicts the label that has higher probability
        label = 0
    else:
        label = 1
    return label

predicted_labels = []
for t in data_test['tweets']:
    predicted_labels.append(predict_label(t))

print(len(predicted_labels))


120


In [7]:
print(list_metrics()) # list of available metrics

metric_accuracy = load_metric('accuracy')
metric_f1 = load_metric('f1')
metric_precision = load_metric('precision')
metric_recall = load_metric('recall')


['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'chrf', 'code_eval', 'comet', 'competition_math', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'gleu', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'pearsonr', 'perplexity', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'wer', 'wiki_split', 'xnli', 'xtreme_s']


In [8]:
#uncomment for info on usage

#print(metric_accuracy)
#print(metric_f1)
#print(metric_precision)
#print(metric_recall)

In [9]:
print(data_test)

                                                tweets  labels
446  Come on, people.\nYou can’t be unhappy when th...     1.0
404  RT @DrEricDing: Can’t believe @GovRonDeSantis ...     1.0
509  Total Covid Death Distribution by County  For ...     0.0
455  RT @njoyflyfishing: Total Covid Death Distribu...     0.0
201  Students of history should have foreseen Russi...     0.0
..                                                 ...     ...
5    RT @njoyflyfishing: Total Covid Death Distribu...     0.0
224  The truth is in this report, and we have been ...     1.0
159  RT @awamkisena: Back to School #Quest for educ...     0.0
38   If you've got #Covid symptoms, PCR testing sit...     0.0
192  RT @the_hindu: #China is seeing a new surge in...     1.0

[120 rows x 2 columns]


In [10]:
refs = data_test['labels']
preds = predicted_labels
results = [metric_accuracy.compute(references=refs, predictions=preds),
           metric_f1.compute(references=refs, predictions=preds),
           metric_precision.compute(references=refs, predictions=preds, average='binary'),
           metric_recall.compute(references=refs, predictions=preds, average='binary')]


In [11]:
from tabulate import tabulate

headers = []
values = []
for i in results:
    for key in i.keys():
        headers.append(key)
        values.append(i[key])
print(tabulate([values], headers=headers))


  accuracy        f1    precision    recall
----------  --------  -----------  --------
  0.858333  0.738462     0.827586  0.666667
